<div style="margin:0;">
    <div style="background-color: lightgray; border: 1px solid Black;  width: 85%; padding: 30px; margin:auto; text-align: center; border-radius: 10px 0; font-weight: bold;">
        <h1 style="text-align:center">Análise Exploratório dos Dados da Covid-19 no Brasil</h1>
    </div>
</div>

<h2>1. Contexto</h2>
<p style="text-align:justify"> Conforme o <a href="https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus">Ministério da Saúde</a>, a Covid-19 é uma doença infecto respiratória causada pelo vírus da SARS-CoV-2, que apresenta elevado potencial de transmissibilidade de impacto global. Que afetou o mundo todo, e gerou mudanças de comportamento a respeito da precaução/proteção de doenças infectocontagiosas.</p>
<p style="text-align:justify">Por esse motivo, o estudo dos dados de transmissão e evolução dos casos em uma região, é fundamental para garantir o combate e controle da doença. Essa projeto visa construir um dashboard de visualização interativa, a respeito dos dados de vacinação e casos de Covid-19 no Brasil. Para esses dados serem consumidos, será utilizado o método de ETL (Extração - Transformação - Carregamento), e depois visualizados na plataforma Google Data Studio.</p>
<p style="text-align:justify">Para fazer isso, serão utilizadas duas bases de dados, a primeira vai conter informações sobre os casos da COVID-19, e a segunda informações de registro das vacinações. A primeira base (Casos de Covid-19), trata-se de um compilado de informações coletadas diariamente desde janeiro de 2020 pelo centro de ciência e engenharia da universidade <a href="https://www.jhu.edu/">John Hopkins</a>, que apresenta descrições geográficas de regiões de países (como estados, condados, províncias, etc.), além disso, seus dados podem ser consultados em seu <a href="https://systems.jhu.edu/research/public-health/ncov/">portal</a>, ou <a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports">repositório</a> público. A segunda base de dados (Registro de Vacinação), refere-se a um compilado desenvolvido pelo projeto Nosso Mundo em Dados da universidade de <a href="https://www.ox.ac.uk/">Oxford</a>, que iniciou em janeiro de 2020, sendo atualizado diariamente com características geográficas de países, os dados podem ser acessados pelo <a href="https://ourworldindata.org">portal - Our World in Data</a>, ou através desse <a href="https://covid.ourworldindata.org/data/owid-covid-data.csv">link</a>.</p>
<!--><p><strong>Problematica:</strong> O efeito da vacinação gerou redução no número de casos no cenário brasileiro?</p><-->
<h3>Descrição dos dados:</h3>
<details open>
    <summary><strong>Caso de COVID-19:</strong></summary>
    <ul>
         <li><em>date</em> = data de referência;
         <li><em>state</em> = estado;
         <li><em>country</em>: país; 
         <li><em>population</em> = população estimada;
         <li><em>confirmed</em> = número acumulado de infectados;
         <li><em>confirmed_1d</em> = número diário de infectados;
         <li><em>confirmed_moving_avg_7d</em> = média móvel de 7 dias do número diário de infectados;
         <li><em>confirmed_moving_avg_7d_rate_14d</em> = média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
         <li><em>deaths</em> = número acumulado de mortos;
         <li><em>deaths_1d</em> = número diário de mortos;
         <li><em>deaths_moving_avg_7d</em> = média móvel de 7 dias do número diário de mortos;
         <li><em>deaths_moving_avg_7d</em> = média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
         <li><em>month</em> = mês de referência;
         <li><em>year</em> = ano de referência.
    </ul>
</details>
<details>
    <summary><strong>Registros de Vacinação da COVID-19:</strong></summary>
    <ul>
        <li><em>date</em> = data de referência;
        <li><em>country</em> = país;
        <li><em>population</em> = população estimada;
        <li><em>total</em> = número acumulado de doses administradas;
        <li><em>one_shot</em> = número acumulado de pessoas com uma dose;
        <li><em>one_shot_perc</em> = número acumulado relativo de pessoas com uma dose;
        <li><em>two_shots</em> = número acumulado de pessoas com duas doses;
        <li><em>two_shot_perc</em> = número acumulado relativo de pessoas com duas doses;
        <li><em>three_shots</em> = número acumulado de pessoas com três doses;
        <li><em>three_shot_perc</em> = número acumulado relativo de pessoas com três doses;
        <li><em>month</em> = mês de referência;
        <li><em>year</em> = ano de referência.
    </ul>
</details>

<h2>2. Pacotes e bibliotecas</h2>


In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

<h2>3. Extração</h2>

In [8]:
#Carregando o DataFrame do Registro de vacinação:
df_vacinacao = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


<p>Como os dados referente aos Casos de Covid-19 são registrados por dia em seu repositório, ou seja, existem vários arquivos com granularidade de cada dia. Será preciso fazer múltiplas extrações para montar um DataFrame com intervalo de tempo equivale ao DataFrame de Registro de Vacinação.</p>
<p>Primeiro, será preciso definir um intervalo de tempo. Para fazer isso, vou consultar a data mais recente do DataFrame de Vacinação usando o método de consulta <code>.query()</code> do Pandas, e depois compara o seu resultado com a data mais recente registrada no repositório de Casos de Covid, o qual é 05 de Dezembro de 2022:</p>

<details>
    <summary>print do repositório</summary>
<img src="https://github.com/SergioAdauto/da-covid-brazil/blob/main/image/print.png?raw=true" width="80%">
</details>

In [25]:
# Observando a data mais recente do DataFrame de vacinação:
display(str(df_vacinacao.query('location == "Brazil"')['date'].values[-1]))

'2022-12-15T00:00:00.000000000'

In [31]:
# Função responsável por definir um intervalo de tempo:
def intervalo_tempo(data_inicio: datetime, data_fim: datetime) -> Iterator[datetime]:
  intervalo_tempo_dias: int = (data_fim - data_inicio).days
  for data in range(intervalo_tempo_dias):
    yield data_inicio + timedelta(data)

In [32]:
# Definindo o intervalo de tempo:
data_inicio = datetime(2021,  1,  1)
data_fim = datetime(2022,  12,  5)

In [34]:
#Variável para registrar o Dataframe de casos de covid-19:
df_casos = None

casos_vazio = True # Variável responsável por validar se o dataframe está concluido.

#Loop para extrair os dados dados do repositório e selecionar as colunas desejadas:
for data in intervalo_tempo(data_inicio, data_fim):
    # Variável responsável por consultar a data desejada no repositório:  
    data_consulta = data.strftime('%m-%d-%Y')
    # link do repositório:
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{data_consulta}.csv'
    # Carregando os dados de casos do dia em um dataframe:
    caso = pd.read_csv(filepath_or_buffer=url, sep=',')
    # Apagando as colunas que não serão utilizadas:
    caso = caso.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
    # Selecionando as informações do Brasil:
    caso = caso.query('Country_Region == "Brazil"').reset_index(drop=True)
    # Convertendo a coluna 'Date' para o formato de data:
    caso['Date'] = pd.to_datetime(data.strftime('%Y-%m-%d'))
    #Verificando se o dataframe está completo: 
    if casos_vazio:
        df_casos = caso
        casos_vazio = False
    else:
        df_casos = df_casos.append(caso, ignore_index=True)

In [36]:
df_casos.tail()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
18976,Roraima,Brazil,179423,2176,29619.437369,2022-12-04
18977,Santa Catarina,Brazil,1910186,22465,26660.746975,2022-12-04
18978,Sao Paulo,Brazil,6197738,176356,13497.095726,2022-12-04
18979,Sergipe,Brazil,348977,6449,15181.520305,2022-12-04
18980,Tocantins,Brazil,346775,4208,22047.332703,2022-12-04


<h2>4. Transformação</h2>

<h4>4.1 Transformação dos dados de caso:</h4>

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
30588,BRA,South America,Brazil,2022-12-11,35531716.0,0.0,27738.571,690677.0,0.0,86.143,...,17.9,NaN,2.2,75.88,0.765,215313504.0,NaN,NaN,NaN,NaN
30589,BRA,South America,Brazil,2022-12-12,35643770.0,112054.0,35368.429,691015.0,338.0,112.286,...,17.9,NaN,2.2,75.88,0.765,215313504.0,NaN,NaN,NaN,NaN
30590,BRA,South America,Brazil,2022-12-13,35696918.0,53148.0,42961.000,691178.0,163.0,135.571,...,17.9,NaN,2.2,75.88,0.765,215313504.0,NaN,NaN,NaN,NaN
30591,BRA,South America,Brazil,2022-12-14,35751411.0,54493.0,50745.714,691449.0,271.0,174.286,...,17.9,NaN,2.2,75.88,0.765,215313504.0,NaN,NaN,NaN,NaN
30592,BRA,South America,Brazil,2022-12-15,35809832.0,58421.0,39730.857,691652.0,203.0,139.286,...,17.9,NaN,2.2,75.88,0.765,215313504.0,NaN,NaN,NaN,NaN
